
## Basic Neural Network Sample
Working with  **skorch**. Consult with the [documentation](https://skorch.readthedocs.io/en/stable/).

# Installing skorch and loading libraries

In [1]:
import subprocess

# Installation on Google Colab
try:
    import google.colab
    subprocess.run(['python', '-m', 'pip', 'install', 'skorch'])
except ImportError:
    pass

In [2]:
import torch
from torch import nn
import torch.nn.functional as F
from skorch import NeuralNetClassifier

In [3]:
#Check if torch works with version verification
print(torch.__version__)

2.1.0


In [4]:
torch.manual_seed(0)
torch.cuda.manual_seed(0)

In [5]:
import pandas as pd
import numpy as np
import csv
import re
import string
from collections import defaultdict

## Training a classifier and making predictions

In [6]:
with open(f'x_train.txt', encoding = 'utf-8') as f:
    x_train = f.read().splitlines()
with open(f'y_train.txt', encoding = 'utf-8') as f:
    y_train = f.read().splitlines()
with open(f'x_test.txt', encoding = 'utf-8') as f:
    x_test = f.read().splitlines()
with open(f'y_test.txt', encoding = 'utf-8') as f:
    y_test = f.read().splitlines()

In [7]:
import pandas as pd
# combine x_train and y_train into one dataframe
train_df = pd.DataFrame({'text': x_train, 'label': y_train})

#combine x_test and y_test into one dataframe
test_df = pd.DataFrame({'text': x_test, 'label': y_test})

In [8]:
# T: Please use again the train/test data that includes English, German, Dutch, Danish, Swedish and Norwegian, plus 20 additional languages of your choice (the labels can be found in the file labels.csv)
# and adjust the train/test split if needed

##Re-utilizing the saved .csv dataset we build in part 1
# combine x_train and y_train into one dataframe
train_df = pd.DataFrame({'text': x_train, 'label': y_train})
# write train_df to csv with tab as separator
train_df.to_csv('train_df_second.csv', index=False, sep='\t')
# combine x_test and y_test into one dataframe
test_df = pd.DataFrame({'text': x_test, 'label': y_test})
# write test_df to csv with tab as separator
test_df.to_csv('test_df_second.csv', index=False, sep='\t')

In [9]:
## Checking the dataset... it works, still the same a the previous one
train_df.head()

,text,label
0,Klement Gottwaldi surnukeha palsameeriti ning ...,est
1,"Sebes, Joseph; Pereira Thomas (1961) (på eng)....",swe
2,भारतीय स्वातन्त्र्य आन्दोलन राष्ट्रीय एवम क्षे...,mai
3,"Après lo cort periòde d'establiment a Basilèa,...",oci
4,ถนนเจริญกรุง (อักษรโรมัน: Thanon Charoen Krung...,tha


In [10]:
## Checking the test set... fine as well
test_df.head()

,text,label
0,Ne l fin de l seclo XIX l Japon era inda çconh...,mwl
1,Schiedam is gelegen tussen Rotterdam en Vlaard...,nld
2,"ГIурусаз батальонал, гьоркьор гIарадабиги лъун...",ava
3,ರಾಜ್ಯಶಾಸ್ತ್ರದ ಪಿತಾಮಹೆ ಅರಿಸ್ಟಾಟಲ್. ರಾಜ್ಯಶಾಸ್ತ್ರ...,tcy
4,Halukum adalah kelenjar tiroid nang menonjol d...,bjn


In [11]:
## Apply splitting:
from sklearn.model_selection import train_test_split

#Concatenate first the test and training
whole_df = pd.concat((train_df, test_df))
#Now we create a mixed set for training and testing:
new_train_df, new_test_df = train_test_split(whole_df, test_size=0.2, stratify=whole_df['label'])

In [12]:
##Repeat:
#Full subset with the 6 target languages and the 20 plus
same_languages_filter = ['eng', 'deu', 'nld', 'dan', 'swe', 'nno', 'est',
                      'zea', 'hat', 'tam', 'pan', 'ton', 'bod', 'arg', 
                      'jpn', 'tur', 'pol', 'mkd', 'ace', 'bak', 'hun', 
                      'ukr', 'san', 'nap', 'lug', 'sun']
print(len(same_languages_filter))

26


In [13]:
## Split a test and training set with the new labels
newdataframe_sub_training = new_train_df[new_train_df['label'].isin(same_languages_filter)]
newdataframe_sub_testing = new_test_df[new_test_df['label'].isin(same_languages_filter)]

In [14]:
## T: use your adjusted code to encode the labels here ##
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer

# Fitting the labels:
le_fitted = LabelEncoder().fit(newdataframe_sub_training['label'])

# Encoding the labels for training and test
y_train_sub = le_fitted.transform(newdataframe_sub_training['label'])
y_test_sub = le_fitted.transform(newdataframe_sub_testing['label'])

In [15]:
# Transform both text, re-using character analyzer:
# Changed max features to 10 for speed:
vectorization = CountVectorizer(analyzer='char', ngram_range=(2, 2), max_features=100, binary=True)
X_main = vectorization.fit_transform(newdataframe_sub_training['text'].to_numpy())
X_test = vectorization.transform(newdataframe_sub_testing['text'].to_numpy())

In [16]:
# Reformat the values into np.float32 and int64 for the other kit:
X_main = X_main.astype(np.float32)
X_test = X_test.astype(np.float32)
y_train_sub = y_train_sub.astype(np.int64)
y_test_sub = y_test_sub.astype(np.int64)

In [17]:
# T: In the following, you can find a small (almost) working example of a neural network. Unfortunately, again, the cat messed up some of the code. Please fix the code such that it is executable.
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC

In the following, we define a vanilla neural network with two hidden layers. The output layer should have as many outputs as there are classes. In addition, it should have a nonlinearity function.

In [18]:
class ClassifierModule(nn.Module):
    def __init__(
            self,
            num_units=200,
            nonlin=F.relu,
    ):
        super(ClassifierModule, self).__init__()
        self.num_units = num_units
        self.nonlin = nonlin

        self.dense0 = nn.Linear(100, num_units)
        self.nonlin = nonlin
        self.dense1 = nn.Linear(num_units, 200)
        self.output = nn.Linear(200, 100)

    def forward(self, X, **kwargs):
      X = self.nonlin(self.dense0(X))
      X = F.relu(self.dense1(X))
      X = self.output(X)
      return X.squeeze(dim=1)

In [19]:
## Vanilla configuration - Baseline
net = NeuralNetClassifier(
    ClassifierModule,
    train_split=None,
    max_epochs=20,
    criterion=nn.CrossEntropyLoss(),
    lr=0.1,
    #device='cuda'  # comment this to train with CPU
)
# we tried to change to lower learning rate but we got Out of memory error. The only thing we could change is the epoch number,
# but after 6 epoch it doesn't seem to improve anymore

In [20]:
net.fit(X_main, y_train_sub)

C:\Users\Argent\anaconda3\lib\site-packages\huggingface_hub\utils\_runtime.py:184: UserWarning: Pydantic is installed but cannot be imported. Please check your installation. `huggingface_hub` will default to not using Pydantic. Error message: '{e}'
  warnings.warn(


  epoch    train_loss     dur
-------  ------------  ------
      1        3.0191  2.8903
      2        1.3631  2.8858
      3        0.9827  2.7074
      4        0.8379  2.7412
      5        0.7748  2.8119
      6        0.7395  2.9261
      7        0.7156  3.0651
      8        0.6980  3.0047
      9        0.6840  2.8019
     10        0.6726  2.7664
     11        0.6628  2.8434
     12        0.6544  2.8364
     13        0.6470  2.7809
     14        0.6403  2.9263
     15        0.6343  3.3957
     16        0.6286  2.9422
     17        0.6235  2.8311
     18        0.6187  3.0893
     19        0.6142  2.8021
     20        0.6098  2.7124


<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=ClassifierModule(
    (dense0): Linear(in_features=100, out_features=200, bias=True)
    (dense1): Linear(in_features=200, out_features=200, bias=True)
    (output): Linear(in_features=200, out_features=100, bias=True)
  ),
)

In [21]:
## Final score for baseline:
net.score(X_test, y_test_sub)

## Score is 0.7325 which isn't bad per se
### Required to get at least 0.8...

0.7325

In [40]:
# grid search for hyperparamter tuning
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

## Replicate Pipeline needed:
pipeline = Pipeline([
    ('vect', CountVectorizer(analyzer='char', ngram_range=(2, 2), max_features=100, binary=True, dtype=np.float32)),
    ('clf', NeuralNetClassifier(
        ClassifierModule,
        criterion=nn.CrossEntropyLoss(),
        lr=0.1,
        #device='cuda'  # comment this to train with CPU
        ))
    ])

In [41]:
## Parameters to check through in the grid for all combinations
## Values are restricted since we are close to 0.80 score needed:
param_grid = {
    "vect__ngram_range": [(1, 1), (2, 2)], # CountVectorizer features
    "clf__module__num_units": [150, 200], # two layer sizes to experiment, just small increments since its almost 0.8
    "clf__module__nonlin": [F.relu, F.tanh], # activation functions
    "clf__optimizer": [torch.optim.Adam, torch.optim.SGD], # optimizers are SGD and ADAM
    "clf__max_epochs": [15, 20], # early stopping for terminate training earlier in order to find best performance at lowest amount of training
}

In [23]:
# Execute the grid check:
## CV at 2 for fastest results and being the minimum k-fold number one can use...
checkgrid = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=2)
grid_best_results = checkgrid.fit(newdataframe_sub_training['text'], y_train_sub)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        3.4657       0.5457        1.5163  1.9921
      2        1.2959       0.6750        1.1942  1.8810
      3        1.0224       0.6885        1.1840  1.8906
      4        1.0677       0.7587        0.9398  1.8120
      5        0.9158       0.7505        0.9484  1.6686
      6        0.8787       0.7308        0.9922  1.6894
      7        0.9072       0.7303        0.9826  1.7822
      8        0.8837       0.7471        1.0866  1.5959
      9        0.8684       0.7524        2.1568  1.6510
     10        1.0723       0.7678        4.8708  1.6622
     11        1.1232       0.6827        6.8702  1.7058
     12        1.0223       0.7582        2.5807  2.5601
     13        0.9337       0.7538        2.5224  2.9479
     14        0.9171       0.7663        2.5232  3.3136
     15        0.8116       0.7865        2.6678  2.8655
  epoch    train_loss    valid_

      9        0.8252       0.7000        0.8462  1.4381
     10        0.7897       0.7063        0.8211  1.3302
     11        0.7631       0.7106        0.8041  1.5399
     12        0.7422       0.7135        0.7919  1.5121
     13        0.7250       0.7149        0.7824  1.4155
     14        0.7103       0.7149        0.7753  1.2850
     15        0.6975       0.7192        0.7697  1.3125
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        3.6867       0.0385        4.0820  1.4881
      2        2.8342       0.2053        3.1777  1.3972
      3        1.9031       0.2740        2.3162  1.2836
      4        1.4500       0.3418        2.0185  1.4399
      5        1.2400       0.4000        1.7302  1.3993
      6        1.0962       0.4702        1.4851  1.4572
      7        0.9872       0.5361        1.2955  1.4356
      8        0.9066       0.5731        1.1586  1.3150
      9        0.8499       0.5

      2        2.7574       0.1437        3.2862  1.4114
      3        2.2117       0.2861        2.4634  1.4096
      4        1.4969       0.4736        1.5687  1.4647
      5        0.9731       0.6120        1.0661  1.3434
      6        0.7193       0.7389        0.8127  1.3368
      7        0.5878       0.7923        0.6707  1.3613
      8        0.5108       0.8163        0.5862  1.2995
      9        0.4613       0.8404        0.5336  1.5692
     10        0.4271       0.8505        0.4988  1.6626
     11        0.4016       0.8548        0.4730  1.4693
     12        0.3815       0.8587        0.4534  1.3152
     13        0.3649       0.8625        0.4375  1.4745
     14        0.3511       0.8673        0.4249  1.3021
     15        0.3393       0.8702        0.4142  1.3981
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        3.7305       0.0486        3.2363  1.3119
      2        2.8512       0.3

     13        3.2731       0.0385        3.2700  1.9483
     14        3.2730       0.0385        3.2700  1.8767
     15        3.2730       0.0385        3.2699  1.9098
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        3.6156       0.1976        2.9134  1.3744
      2        2.6548       0.3125        2.3863  1.3273
      3        2.0640       0.6380        1.7142  1.4730
      4        1.3609       0.7937        1.0605  1.3785
      5        0.8757       0.8438        0.7326  1.3221
      6        0.6465       0.8606        0.5780  1.4192
      7        0.5293       0.8659        0.4957  1.3205
      8        0.4616       0.8750        0.4477  1.3752
      9        0.4188       0.8817        0.4175  1.3769
     10        0.3895       0.8875        0.3970  1.2717
     11        0.3681       0.8899        0.3821  1.3126
     12        0.3516       0.8904        0.3710  1.3611
     13        0.3383       0.8

      5        2.9256       0.0913        2.8323  1.5029
      6        2.7417       0.1149        2.6881  1.6554
      7        2.6843       0.1293        2.6618  1.5159
      8        2.6825       0.1212        2.6356  1.5484
      9        2.9597       0.0385        3.7363  1.6363
     10        3.3507       0.0385        3.2688  1.5341
     11        3.2747       0.0385        3.2687  1.6672
     12        3.2743       0.0385        3.2683  2.0048
     13        3.2740       0.0385        3.2681  2.5144
     14        3.2739       0.0385        3.2679  2.3003
     15        3.2738       0.0385        3.2678  2.2463
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        4.4944       0.0880        2.9283  1.4967
      2        2.8409       0.0885        2.8226  1.4041
      3        2.8037       0.0894        2.8126  1.4095
      4        2.7884       0.1010        2.7376  1.5015
      5        3.1502       0.0

...

     18        2.0894       0.2524        2.0684  3.0586
     19        2.0950       0.2279        2.1491  3.0663
     20        2.0811       0.2466        2.1517  2.9913
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        4.9256       0.1125        2.8326  1.6999
      2        2.6616       0.1327        2.6793  1.5708
      3        2.6338       0.1375        2.5973  1.4455
      4        2.5970       0.1442        2.6113  1.6433
      5        2.5819       0.1413        2.6076  1.5344
      6        2.5685       0.1279        2.5662  1.5506
      7        2.5639       0.1293        2.5816  1.5688
      8        2.5667       0.1226        2.6305  1.6480
      9        2.5596       0.1471        2.5990  1.6499
     10        2.5569       0.1346        2.5960  1.7449
     11        2.5693       0.1202        2.6295  1.8002
     12        2.5609       0.1212        2.6078  2.5551
     13        2.5566       0.1

     14        0.7208       0.6721        0.8641  1.2652
     15        0.7075       0.6745        0.8514  1.3454
     16        0.6959       0.6779        0.8406  1.3303
     17        0.6856       0.6880        0.8321  1.3177
     18        0.6765       0.6913        0.8247  1.2393
     19        0.6681       0.6952        0.8190  1.4055
     20        0.6606       0.6971        0.8146  1.3161
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        3.7187       0.1135        2.8457  1.4022
      2        2.7921       0.1524        2.8529  1.4622
      3        2.6686       0.2005        2.6142  1.5270
      4        2.6622       0.1784        2.6025  1.6710
      5        2.6354       0.2226        2.5730  1.4856
      6        2.5068       0.1678        2.5443  1.7768
      7        2.7370       0.1562        2.7771  1.4479
      8        2.7363       0.1697        2.7520  1.8913
      9        2.7212       0.1

      3        2.0130       0.3500        2.0140  1.3442
      4        1.3484       0.5591        1.2501  1.3347
      5        0.8931       0.7308        0.8693  1.2728
      6        0.6718       0.7957        0.6839  1.4259
      7        0.5562       0.8207        0.5826  1.3857
      8        0.4881       0.8385        0.5212  1.3556
      9        0.4436       0.8447        0.4805  1.2764
     10        0.4121       0.8538        0.4518  1.3822
     11        0.3884       0.8630        0.4301  1.3883
     12        0.3697       0.8697        0.4134  1.2391
     13        0.3544       0.8736        0.3999  1.5138
     14        0.3417       0.8779        0.3888  1.4044
     15        0.3309       0.8832        0.3794  1.3855
     16        0.3217       0.8856        0.3716  1.2499
     17        0.3137       0.8880        0.3650  1.4813
     18        0.3068       0.8894        0.3594  1.3024
     19        0.3006       0.8928        0.3546  1.2639
     20        0.2952       0.8

      3        3.2805       0.0385        3.2716  1.5497
      4        3.2773       0.0385        3.2695  1.4555
      5        3.2759       0.0385        3.2684  1.4148
      6        3.2751       0.0385        3.2678  1.4860
      7        3.2747       0.0385        3.2674  1.4812
      8        3.2744       0.0385        3.2672  1.4307
      9        3.2742       0.0385        3.2670  1.6399
     10        3.2741       0.0385        3.2669  1.5485
     11        3.2740       0.0385        3.2668  1.4949
     12        3.2740       0.0385        3.2668  2.2313
     13        3.2739       0.0385        3.2668  2.8619
     14        3.2739       0.0385        3.2668  3.1059
     15        3.2739       0.0385        3.2667  2.7649
     16        3.2739       0.0385        3.2667  2.9688
     17        3.2739       0.0385        3.2667  2.8690
     18        3.2739       0.0385        3.2668  2.9801
     19        3.2739       0.0385        3.2668  2.9178
     20        3.2739       0.0

      3        1.7157       0.3591        1.8240  1.3844
      4        1.3434       0.4591        1.5046  1.4005
      5        1.1394       0.5346        1.2991  1.3182
      6        1.0048       0.5740        1.1538  1.4973
      7        0.9156       0.6034        1.0547  1.3818
      8        0.8553       0.6231        0.9854  1.3466
      9        0.8125       0.6418        0.9362  1.4185
     10        0.7806       0.6548        0.9005  1.3545
     11        0.7560       0.6649        0.8736  1.4417
     12        0.7363       0.6774        0.8531  1.3541
     13        0.7201       0.6861        0.8373  1.2621
     14        0.7065       0.6894        0.8244  1.2608
     15        0.6947       0.6918        0.8140  1.3429
     16        0.6843       0.6942        0.8054  1.4108
     17        0.6750       0.6981        0.7981  1.2755
     18        0.6666       0.7014        0.7922  1.5495
     19        0.6590       0.7067        0.7870  1.3393
     20        0.6520       0.7

In [42]:
## Check the best results:
print('Best parameters given by GridSearchCV are:', grid_best_results.best_params_)

Best parameters given by GridSearchCV are: {'clf__max_epochs': 20, 'clf__module__nonlin': <function tanh at 0x000001A4B1031EE0>, 'clf__module__num_units': 200, 'clf__optimizer': <class 'torch.optim.sgd.SGD'>, 'vect__ngram_range': (1, 1)}


In [43]:
# show results of optimized vectorizer and net on test set

optimized_pipe = grid_best_results.best_estimator_
optimized_pipe.score(newdataframe_sub_testing['text'], y_test_sub)

## 0.9126923076923077 DONE ###

0.9126923076923077